<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PostgreSQL_04B_Ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises
###Adapted fromMySQL Tutorials <br>


1.   https://www.mysqltutorial.org/
2.   https://www.mysqltutorial.org/basic-mysql-tutorial.aspx


#Install & Configure Postgres

In [1]:
from datetime import datetime
import pytz
print('Tested on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Tested on  2024-01-07 17:04:52.460981+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


For Connecting to a Postgres server on Amazon RDS, see [this notebook](https://github.com/Praxis-QR/RDWH/blob/main/PostgreSQL_01_Install_Config.ipynb)

In [4]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!pip install psycopg2
import psycopg2
import pandas as pd
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


#Create Schema, Tables and Insert Data

In [5]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
#!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:postgres:pass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:upraxis:upass
localhost:5432:dbpraxis:postgres:pass


In [6]:
# Download SQL file to create tables
!wget -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/classic42.sql
#!unzip mysqlsampledatabase.zip
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -f classic42.sql -x -q

psql:classic42.sql:3: NOTICE:  table "customers" does not exist, skipping
psql:classic42.sql:151: NOTICE:  table "employees" does not exist, skipping
psql:classic42.sql:198: NOTICE:  table "offices" does not exist, skipping
psql:classic42.sql:226: NOTICE:  table "orderdetails" does not exist, skipping
psql:classic42.sql:3242: NOTICE:  table "orders" does not exist, skipping
psql:classic42.sql:3589: NOTICE:  table "payments" does not exist, skipping
psql:classic42.sql:3879: NOTICE:  table "productlines" does not exist, skipping
psql:classic42.sql:3902: NOTICE:  table "products" does not exist, skipping


![alt text](https://www.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)<br>

In [7]:
#Note eight tables
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c "\dt+"

                                        List of relations
 Schema |     Name     | Type  |  Owner  | Persistence | Access method |    Size    | Description 
--------+--------------+-------+---------+-------------+---------------+------------+-------------
 public | customers    | table | upraxis | permanent   | heap          | 48 kB      | 
 public | employees    | table | upraxis | permanent   | heap          | 8192 bytes | 
 public | offices      | table | upraxis | permanent   | heap          | 8192 bytes | 
 public | orderdetails | table | upraxis | permanent   | heap          | 232 kB     | 
 public | orders       | table | upraxis | permanent   | heap          | 64 kB      | 
 public | payments     | table | upraxis | permanent   | heap          | 48 kB      | 
 public | productlines | table | upraxis | permanent   | heap          | 16 kB      | 
 public | products     | table | upraxis | permanent   | heap          | 72 kB      | 
(8 rows)



In [8]:
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from customers limit 10"
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from employees limit 10"
#psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from offices limit 10"
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from orderdetails limit 10"
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from orders limit 10"
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from payments limit 10"
#!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from productlines limit 10"
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from products limit 10"

 productcode |              productname              |   productline    | productscale |       productvendor       |                                                                                                                                                                                                                                              productdescription                                                                                                                                                                                                                                               | quantityinstock | buyprice |  msrp  
-------------+---------------------------------------+------------------+--------------+---------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [9]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [10]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [11]:
runSelect("select * from orders limit 3")

ordernumber,orderdate,requireddate,shippeddate,status,comments,customernumber
10100,2003-01-06,2003-01-13,2003-01-10,Shipped,None,363
10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
10102,2003-01-10,2003-01-18,2003-01-14,Shipped,None,181


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2023-12-31 16:01:25.642107+05:30
